In [1]:
!git clone https://github.com/DevoLearn/DevoGraph.git

Cloning into 'DevoGraph'...
remote: Enumerating objects: 3246, done.
remote: Counting objects: 100% (2003/2003), done.
remote: Compressing objects: 100% (1950/1950), done.
remote: Total 3246 (delta 96), reused 1928 (delta 50), pack-reused 1243
Receiving objects: 100% (3246/3246), 233.15 MiB | 22.40 MiB/s, done.
Resolving deltas: 100% (178/178), done.
Updating files: 100% (2571/2571), done.


In [2]:
import sys
sys.path.append('/content/DevoGraph')

In [3]:
!pip install  dgl -f https://data.dgl.ai/wheels/torch-2.3/cu121/repo.html

Looking in links: https://data.dgl.ai/wheels/torch-2.3/cu121/repo.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.3/309.3 MB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 17.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.

In [4]:
!pip show torch

Name: torch
Version: 2.3.0+cu121
Summary: Tensors and Dynamic neural networks in Python with strong GPU acceleration
Home-page: https://pytorch.org/
Author: PyTorch Team
Author-email: packages@pytorch.org
License: BSD-3
Location: /usr/local/lib/python3.10/dist-packages
Requires: filelock, fsspec, jinja2, networkx, nvidia-cublas-cu12, nvidia-cuda-cupti-cu12, nvidia-cuda-nvrtc-cu12, nvidia-cuda-runtime-cu12, nvidia-cudnn-cu12, nvidia-cufft-cu12, nvidia-curand-cu12, nvidia-cusolver-cu12, nvidia-cusparse-cu12, nvidia-nccl-cu12, nvidia-nvtx-cu12, sympy, triton, typing-extensions
Required-by: fastai, torchaudio, torchdata, torchtext, torchvision


In [6]:
%cd DevoGraph

/content/DevoGraph


/usr/local/lib/python3.10/dist-packages/torch/utils/data/datapipes/utils/common.py by adding a new line: DILL_AVAILABLE = dill_available(). Then the issue was solved.

In [8]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import networkx as nx
import numpy as np
import torch as th


from copy import deepcopy
from importlib import reload
import devograph.datasets.datasets as data

%matplotlib inline
%config InlineBackend.figure_format='retina'

In [9]:
raw_data = pd.read_csv('/content/DevoGraph/data/CE_raw_data.csv')
# raw_data.drop(['cellTime'], axis=1, inplace=True)
raw_data.drop(raw_data[(raw_data.time == 'time')].index, axis=0, inplace=True)
raw_data

,Unnamed: 0,cell,time,x,y,z,size
0,0,AB,1,454.0,286.0,14.0,53.0
1,1,AB,2,494.0,244.0,14.5,68.0
2,2,AB,3,497.0,235.0,14.0,48.0
3,3,AB,4,491.0,223.0,14.0,50.0
4,4,ABa,5,519.0,272.0,14.3,70.0
...,...,...,...,...,...,...,...
252280,252293,Cppp,87,578.0,205.0,14.1,40.0
252281,252294,Cppp,88,583.0,215.0,14.0,40.0
252282,252295,Cppp,89,573.0,210.0,13.1,38.0
252283,252296,Cpppa,90,562.0,233.0,14.8,36.0


In [10]:
raw_data.time = raw_data.time.astype('int')
raw_data.x = raw_data.x.astype('float')
raw_data.y = raw_data.y.astype('float')
raw_data.z = raw_data.z.astype('float')
raw_data['size'] = raw_data['size'].astype('float')

In [11]:
raw_data.to_csv('/content/CE_lineage_data.csv')

In [17]:
lineage_data = pd.read_excel('/content/cell-by-cell-data-v2.xlsx', sheet_name='daughter-of-database',
                             engine='openpyxl', usecols=['CELL NAME', 'CELL NAME.1'])
lineage_data.rename(columns={'CELL NAME': 'daughter', 'CELL NAME.1': 'mother'}, inplace=True)
lineage_data

,daughter,mother
0,AB,P0
1,ABa,AB
2,ABal,ABa
3,ABala,ABal
4,ABalaa,ABala
...,...,...
1860,Z4.appaaap,Z4.appaaa
1861,Z4.appaap,Z4.appaa
1862,Z4.appap,Z4.appa
1863,Z4.appp,Z4.app


In [18]:
lineage_data.to_csv('/content/CE_lineage_data.csv')

In [27]:
# Download raw data from url then generate temporal graphs based on them.
# The graph building process is based on KNN algorithm.
datasets = data.CETemporalGraphKNN(
        time_start=0, time_end=10, knn_k=3, columns=['size'],
        url='https://raw.githubusercontent.com/LspongebobJH/DevoGraph/main/data/CE_raw_data.csv?token=GHSAT0AAAAAABMX6RJRRFC2U5QOCZXHNBVYYVL5Y2A')

res_g, batch_node_interval = data.to_temporal_directed(datasets, '/content/CE_lineage_data.csv')

# Store the directed temporal graphs and related information into datasets for later operations
datasets.set_batch_graph(res_g)
datasets.set_info({'batch_node_interval': batch_node_interval})

In [20]:
print(f"number of frames: {len(datasets)}")
print(f"number of nodes in the batch graph: {datasets.batch_graph.number_of_nodes()}")
print(f"number of edges in the batch graph: {datasets.batch_graph.number_of_edges()}")

number of frames: 10
number of nodes in the batch graph: 627
number of edges in the batch graph: 5397


In [21]:
!pip install PyQt5

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 338.1/338.1 kB 36.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.5/60.5 MB 12.0 MB/s eta 0:00:00


In [22]:
import numpy

def patch_asscalar(a):
    return a.item()

setattr(numpy, "asscalar", patch_asscalar)

In [23]:
!pip install kmapper

In [29]:
import sys

try:
    import pandas as pd
except ImportError as e:
    print(
        "pandas is required for this example. Please install with `pip install pandas` and then try again."
    )
    sys.exit()

import numpy as np
import kmapper as km
import sklearn
from sklearn import ensemble

# Via:
df = pd.read_csv("/content/CE_lineage_data.csv")
feature_names = [c for c in df.columns if c not in ["daughter", "mother"]]
df["daughter"] = df["daughter"].apply(lambda x: 1 if x == "M" else 0)
X = np.array(df[feature_names].fillna(0))  # quick and dirty imputation
y = np.array(df["daughter"])

# Create a custom 1-D lens with Isolation Forest
model = ensemble.IsolationForest(random_state=1729)
model.fit(X)
lens1 = model.decision_function(X).reshape((X.shape[0], 1))

# Create another 1-D lens with L2-norm
mapper = km.KeplerMapper(verbose=3)
lens2 = mapper.fit_transform(X, projection="l2norm")

# Combine both lenses to create a 2-D [Isolation Forest, L^2-Norm] lens
lens = np.c_[lens1, lens2]

# Create the simplicial complex
graph = mapper.map(
    lens,
    X,
    cover=km.Cover(n_cubes=15, perc_overlap=0.4),
    clusterer=sklearn.cluster.KMeans(n_clusters=2, random_state=1618033),
)

# Visualization
mapper.visualize(
    graph,
    path_html="/content/output/celegans.html",
    title="C.elegans Lineage Tree Dataset",
    custom_tooltips=y,
)


# Visualization with multiple color functions
mapper.visualize(
    graph,
    path_html="/content/output/celegans-multiple-color-functions.html",
    title="C-elegans Lineage Tree Dataset",
    custom_tooltips=y,
    color_values=lens,
    color_function_name=["Isolation Forest", "L2-norm"],
)


# Visualization with multiple node color functions
mapper.visualize(
    graph,
    path_html="/content/output/celegans-multiple-node-color-functions.html",
    title="C-elegans Lineage Tree Dataset",
    custom_tooltips=y,
    node_color_function=["mean", "std", "median", "max"],
)

# Visualization showing both multiple color functions, and also multiple node color functions
mapper.visualize(
    graph,
    path_html="/content/output/celegans-multiple-color-functions-and-multiple-node-color-functions.html",
    title="C-elegans Lineage Tree Dataset",
    custom_tooltips=y,
    color_values=lens,
    color_function_name=["Isolation Forest", "L2-norm"],
    node_color_function=["mean", "std", "median", "max"],
)


import matplotlib.pyplot as plt

km.draw_matplotlib(graph)
plt.show()

KeplerMapper(verbose=3)
..Composing projection pipeline of length 1:
	Projections: l2norm
	Distance matrices: False
	Scalers: MinMaxScaler()
..Projecting on data shaped (1865, 1)

..Projecting data using: l2norm

..Scaling with: MinMaxScaler()

Mapping on data shaped (1865, 1) using lens shaped (1865, 2)

Minimal points in hypercube before clustering: 2
Creating 225 hypercubes.
   > Found 2 clusters in hypercube 0.
   > Found 2 clusters in hypercube 1.
   > Found 2 clusters in hypercube 2.
   > Found 2 clusters in hypercube 3.
   > Found 2 clusters in hypercube 4.
   > Found 2 clusters in hypercube 5.
   > Found 2 clusters in hypercube 6.
   > Found 2 clusters in hypercube 7.
   > Found 2 clusters in hypercube 8.
   > Found 2 clusters in hypercube 9.
   > Found 2 clusters in hypercube 10.
   > Found 2 clusters in hypercube 11.
   > Found 2 clusters in hypercube 12.
   > Found 2 clusters in hypercube 13.
   > Found 2 clusters in hypercube 14.
   > Found 2 clusters in hypercube 15.
   > 

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:8

   > Found 2 clusters in hypercube 32.
Cube_33 is empty.

   > Found 2 clusters in hypercube 34.
   > Found 2 clusters in hypercube 35.
   > Found 2 clusters in hypercube 36.
   > Found 2 clusters in hypercube 37.
   > Found 2 clusters in hypercube 38.
   > Found 2 clusters in hypercube 39.
   > Found 2 clusters in hypercube 40.
   > Found 2 clusters in hypercube 41.
   > Found 2 clusters in hypercube 42.
   > Found 2 clusters in hypercube 43.
   > Found 2 clusters in hypercube 44.
   > Found 2 clusters in hypercube 45.
   > Found 2 clusters in hypercube 46.
   > Found 2 clusters in hypercube 47.
   > Found 2 clusters in hypercube 48.
Cube_49 is empty.

   > Found 2 clusters in hypercube 50.
   > Found 2 clusters in hypercube 51.
   > Found 2 clusters in hypercube 52.
   > Found 2 clusters in hypercube 53.
   > Found 2 clusters in hypercube 54.
   > Found 2 clusters in hypercube 55.
   > Found 2 clusters in hypercube 56.
   > Found 2 clusters in hypercube 57.
   > Found 2 clusters in h

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:8

   > Found 2 clusters in hypercube 65.
   > Found 2 clusters in hypercube 66.
   > Found 2 clusters in hypercube 67.
   > Found 2 clusters in hypercube 68.
   > Found 2 clusters in hypercube 69.
   > Found 2 clusters in hypercube 70.
   > Found 2 clusters in hypercube 71.

Created 271 edges and 140 nodes in 0:00:00.471503.


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:8

Wrote visualization to: /content/output/celegans.html
Wrote visualization to: /content/output/celegans-multiple-color-functions.html
Wrote visualization to: /content/output/celegans-multiple-node-color-functions.html
Wrote visualization to: /content/output/celegans-multiple-color-functions-and-multiple-node-color-functions.html
no display found. Using non-interactive Agg backend
